In [49]:
import time
import random
import csv
import itertools
import json
import re
import os
import math
import pickle
from xgboost import XGBClassifier

import numpy as np
import pennylane as qml
from concurrent.futures import ThreadPoolExecutor

from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.metrics import accuracy_score

# Classical LIME
from lime.lime_text import LimeTextExplainer

# PART 0: DATA LOADING AND PREPROCESSING

In [50]:

def clean_text(text):
    """
    Removes HTML tags and converts to lowercase.
    """
    # Remove anything between <...> tags, then lowercase the text
    cleaned = re.sub(r'<.*?>', '', text).lower()
    return cleaned

def load_imdb_subset(
    num_samples=5000, 
    min_df=1, 
    max_features=15, 
    stopwords_option=True,
    stop_words = 'english'
):
    """
    Loads a subset of IMDb data, returns:
      - X_train, X_test (lists of text)
      - y_train, y_test (0/1 sentiment)
      - vectorizer (CountVectorizer)
    
    Now with text cleaning for HTML, lowercase, etc.
    """
    data = load_files(
        "C:/Users/migue/Downloads/aclImdb_v1/aclImdb/train",
        categories=['pos','neg'], 
        encoding="utf-8", 
        decode_error="replace"                  
    )
    X_text_all, y_all = data.data, data.target

    # Clean text (HTML removal + lowercase)
    X_text_all = [clean_text(txt) for txt in X_text_all]

    # Shuffle & truncate to num_samples
    full_idx = np.arange(len(X_text_all))
    #np.random.shuffle(full_idx)
    subset_idx = full_idx[:num_samples]
    X_text = [X_text_all[i] for i in subset_idx]
    y = y_all[subset_idx]

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_text, y, test_size=0.2, random_state=42
    )

    # Vectorizer: presence/absence
    if stopwords_option:
        vectorizer = CountVectorizer(
            binary=True, stop_words=stop_words, 
            min_df=min_df, max_features=max_features
        )
    else:
        vectorizer = CountVectorizer(
            binary=True, stop_words=None, 
            min_df=min_df, max_features=max_features
        )

    vectorizer.fit(X_train)
    return X_train, X_test, y_train, y_test, vectorizer

#def train_XGBoost_classifier(X_train, y_train, vectorizer):
    """
    Trains an XGBoost classifier on the binary presence/absence of words.
    Returns the fitted model.
    """
    X_train_bow = vectorizer.transform(X_train)
    # Use log(len(y_train)) as n_estimators (rounded to an int)
    clXGB = XGBClassifier(
        #booster="gblinear",
        objective="binary:logistic", 
        eval_metric="logloss", 
        random_state=42, 
        n_estimators=int(round(math.log(len(y_train)))),
        learning_rate=0.1, 
        max_depth=3
    )
    clXGB.fit(X_train_bow, y_train)
    return clXGB

#def get_cached_xgboost(X_train, y_train, vectorizer, num_samples, max_features, stopwords_option):
    """
    Checks if a classifier trained with the given parameters exists.
    If so, load it; otherwise, train it and save it.
    """
    filename = f"cached_xgboost_ns{num_samples}_mf{max_features}_sw{stopwords_option}_xgboost_classifier_seed42.pkl"
    if os.path.exists(filename):
        print("Loading cached xgboost from", filename)
        with open(filename, 'rb') as f:
            clXGB = pickle.load(f)
    else:
        print("No cached classifier found. Training a new one...")
        clXGB = train_XGBoost_classifier(X_train, y_train, vectorizer)
        with open(filename, 'wb') as f:
            pickle.dump(clXGB, f)
        print("Cached classifier saved as", filename)
    return clXGB

def train_logistic_classifier(X_train, y_train, vectorizer):
    """
    Trains a logistic regression on the binary presence/absence of words.
    Returns the fitted model.
    """
    X_train_bow = vectorizer.transform(X_train)
    clf = LogisticRegression()
    clf.fit(X_train_bow, y_train)
    return clf

def get_cached_logistic(X_train, y_train, vectorizer, num_samples, max_features, stop_words):
    """
    Checks if a classifier trained with the given parameters exists.
    If so, load it; otherwise, train it and save it.
    """
    filename = f"cached_classifier_ns{num_samples}_mf{max_features}_sw{stop_words}_logistic_classifier_seed42.pkl"
    if os.path.exists(filename):
        print("Loading cached logistic from", filename)
        with open(filename, 'rb') as f:
            clf = pickle.load(f)
    else:
        print("No cached classifier found. Training a new one...")
        clf = train_logistic_classifier(X_train, y_train, vectorizer)
        with open(filename, 'wb') as f:
            pickle.dump(clf, f)
        print("Cached classifier saved as", filename)
    return clf

#def train_lasso_regression(X_train, y_train, vectorizer):
    """
    Trains a logistic regression on the binary presence/absence of words.
    Returns the fitted model.
    """
    X_train_bow = vectorizer.transform(X_train)
    lasso_model = Lasso(alpha=0.5)
    lasso_model.fit(X_train_bow, y_train)
    return lasso_model
    

#def get_cached_lasso(X_train, y_train, vectorizer, num_samples, max_features, stopwords_option, alpha):
    """
    Checks if a Lasso model trained with the given parameters exists.
    If so, load it; otherwise, train it and save it.
    """
    filename = f"cached_lasso_ns{num_samples}_mf{max_features}_sw{stopwords_option}_seed42_alpha{alpha}.pkl"
    if os.path.exists(filename):
        print("Loading cached Lasso model from", filename)
        with open(filename, 'rb') as f:
            lasso_model = pickle.load(f)
    else:
        print("No cached Lasso model found. Training a new one...")
        lasso_model = train_lasso_regression(X_train, y_train, vectorizer)
        with open(filename, 'wb') as f:
            pickle.dump(lasso_model, f)
        print("Cached Lasso model saved as", filename)
    return lasso_model

# CLASSICAL LIME

In [51]:



#CHANGE clXGB TO clf IF WE WANT LOGISTIC INSTEAD OF XGBOOST
def run_classical_lime(
    text_sample, clXGB, vectorizer, 
    k_features=10, num_samples=500
):
    """
    Runs classical LIME on a single text instance.
    Returns the top (word, weight) pairs.
    """
    class_names = ["negative", "positive"]
    explainer = LimeTextExplainer(class_names=class_names, feature_selection="auto")

    def predict_proba(texts):
        bow = vectorizer.transform(texts) 
        return clXGB.predict_proba(bow)

    explanation = explainer.explain_instance(
        text_sample,
        predict_proba,
        num_features=k_features,
        num_samples=num_samples  # e.g. 300 or 500
    )
    return explanation.as_list()  # list of (word, weight)

# Q-LIME Pi (Flip Only 1->0)

In [ ]:
def classical_classifier(features, weights, bias=0.0, threshold=0.01):
    # Ensure inputs are 1D arrays
    features = np.array(features).flatten()
    weights = np.array(weights).flatten()
    
    # Zero out small weights
    sparse_weights = np.where(np.abs(weights) < threshold, 0, weights)
    score = bias + np.dot(features, sparse_weights)
    return float(1 / (1 + np.exp(-score)))

def encode_and_flip(features, flip_index=None, shots=None):
    """
    Encode features -> quantum circuit.
    FLIP ONLY if bit == 1 at flip_index (1->0).
    """
    num_qubits = len(features)
    dev = qml.device("default.qubit", wires=num_qubits, shots=shots)

    @qml.qnode(dev)
    def circuit():
        for i, f in enumerate(features):
            if i == flip_index: # and f == 1:   ;;  This line is the original code commented out
                # 1->0 => RY(0),
                #theta = 0
                #My suggestion: 
                theta = f * (np.pi / 2)            
                qml.PauliX(wires=i)
            else:
                theta = f * (np.pi / 2)
 
            qml.RY(theta, wires=i)
            
        return qml.probs(wires=range(num_qubits))

    return circuit()

def sample_state(probabilities):
    """
    Sample an integer state index from the distribution.
    """
    r = random.random()
    cumsum = 0.0
    for idx, p in enumerate(probabilities):
        cumsum += p
        if r <= cumsum:
            return idx
    return len(probabilities) - 1

def measure_and_map_to_classical(features, flip_index=None, shots=None):
    """
    Run the circuit, measure, return a binary array for the top-likelihood state.
    """
    probs = encode_and_flip(features, flip_index=flip_index, shots=shots)
    measured_state = sample_state(probs)
    num_qubits = len(features)
    bin_string = f"{measured_state:0{num_qubits}b}"
    return [int(bit) for bit in bin_string]


def quantum_lime_explanation(
    features, weights, bias=0.0, shots=None
):
    """
    Flip only features that are 1 -> 0.
    Return array of shape (n_features,) with:
       Delta f_k = (original_pred - new_pred).
    """
    original_pred = classical_classifier(features, weights, bias=bias)
    contributions = np.zeros(len(features))

    def flip_and_predict(i):
        new_vec = measure_and_map_to_classical(features, flip_index=i, shots=None)
        new_pred = classical_classifier(new_vec, weights, bias=bias)
        return original_pred - new_pred

    # Flip only bits that are 1
    with ThreadPoolExecutor() as executor:
        futures = {
            executor.submit(flip_and_predict, i): i
            for i, val in enumerate(features) #if val == 1 # ;  This is the original code commented out 
        }
        for future in futures:
            i = futures[future]
            contributions[i] = future.result()

    return contributions

#def quantum_lime_explanation(features, clf, lasso, shots=None):
    
    # Reshape features for prediction (ensure 2D array)
    features_reshaped = np.array(features).reshape(1, -1)
    original_pred = clf.predict_proba(features_reshaped)[0, 1]
    contributions = np.zeros(len(features))

    def flip_and_predict(i):
        # Create a new feature vector with feature i flipped from 1 to 0
        new_vec = features.copy()
        new_vec[i] = 0
        new_vec_reshaped = np.array(new_vec).reshape(1, -1)
        new_pred = lasso.predict(new_vec_reshaped)[0]
        return original_pred - new_pred
 
    # Flip only bits that are 1
    with ThreadPoolExecutor() as executor:
        futures = {
            executor.submit(flip_and_predict, i): i
            for i, val in enumerate(features) if val == 1
        }
        for future in futures:
            i = futures[future]
            contributions[i] = future.result()

    return contributions

# EXPERIMENTAL ROUTINE

In [53]:
def run_experiment( #Did I change these numbers? check if i fcked up smth here!!!!!!!!!!
    num_samples=10,
    min_df=1,
    max_features=15,
    stopwords_option=True,
    lime_num_samples=30,
    shots=None,
    n_test_explanations=10,
    stop_words = None
):
    """
    1) Load data with given params (includes text cleaning)
    2) Train logistic classifier
    3) Evaluate test accuracy
    4) Pick n_test_explanations random samples
    5) For each, run classical LIME vs. Q-LIME Pi
    6) Return summary stats
    """
    # A) Load data
    X_train, X_test, y_train, y_test, vectorizer = load_imdb_subset(
        num_samples=num_samples,
        min_df=min_df,
        max_features=max_features,
        stopwords_option=stopwords_option,
        stop_words = stop_words
    )
    # B) Train model
    clf  = get_cached_logistic(X_train, y_train, vectorizer, num_samples, max_features, stopwords_option)

    #clXGB = get_cached_xgboost(X_train, y_train, vectorizer, num_samples, max_features, stopwords_option)

    # Evaluate
    X_test_bow = vectorizer.transform(X_test)
    #test_acc = accuracy_score(y_test, clf.predict(X_test_bow))
    test_acc = accuracy_score(y_test, clf.predict(X_test_bow))

    logistic_weights = clf.coef_[0]
    bias = clf.intercept_[0]


    #IT ONLY GIVES 1 WEIGHT NOT 15
    #logistic_weights = clXGB.coef_[0]
    #bias = clXGB.intercept_[0]


    #lasso_model = get_cached_lasso(X_train, y_train, vectorizer, num_samples, max_features, stopwords_option, alpha=0.1)
    

    # We'll track times & top-feature overlap
    lime_times = []
    qlime_times = []
    overlaps = []
    instance_local_accuracies = []

    # Random samples for explanation
    #n_test = len(X_test)
    sample_indices = [5,6,12,11,10, 0, 1, 2, 3, 4]
    #random.sample(range(n_test), n_test_explanations)

    for idx in sample_indices:
        text_sample = X_test[idx]
        y_true = y_test[idx]

        # 1) Classical LIME
        start_lime = time.time()
        explanation_lime = run_classical_lime(
            text_sample, clf, vectorizer, 
            k_features=15, num_samples=lime_num_samples
        )

        bow = vectorizer.transform([text_sample])
        bin_features = bow.toarray()[0]

        y_pred = clf.predict(bow)[0]
        instance_accuracy = int(y_pred == y_true)
        instance_local_accuracies.append(instance_accuracy)

        #explanation_lime = run_classical_lime(
        #    text_sample, clf, vectorizer, 
        #    k_features=15, num_samples=lime_num_samples
        #)
        lime_time = time.time() - start_lime
        lime_times.append(lime_time)

        # parse top features
        lime_dict = dict(explanation_lime)
        top_words_lime = sorted(
            lime_dict.keys(),
            key=lambda w: abs(lime_dict[w]),
            reverse=True
        )[:5]

        # 2) Q-LIME Pi
        

        start_qlime = time.time()
        #contributions_qlime = quantum_lime_explanation(bin_features, clf, lasso_model, shots=shots)
        contributions_qlime = quantum_lime_explanation(
            bin_features, logistic_weights, bias=bias, shots=shots)


        contributions_lime_abs = [(word, abs(score)) for word, score in explanation_lime] # Absolute values for comparison; This is a tuple. PROB SHOULD MAKE QLIME A TUPLE TOO!
        
        unsorted_contributions_qlime_abs = tuple(
            (word, abs(score)) for word, score in zip(vectorizer.get_feature_names_out(), contributions_qlime)) # Absolute values for comparison
        
        contributions_qlime_sorted = tuple(
        sorted(unsorted_contributions_qlime_abs, key=lambda x: x[1], reverse=True))

        #print("X_test_bow",X_test_bow)
          
        print("text sample", text_sample, "bin_features", bin_features)
        #, "vectorizer", vectorizer.get_feature_names_out(), "contributions_qlime_abs", contributions_qlime_abs, "Contributions_Lime", top_words_lime
     
        print("Classical LIME Explanation:")
        for word, weight in contributions_lime_abs:
            print(f"Word: {word}, Importance: {weight}")

        print("\nQ-LIME Pi Explanation:")
        for word, weight in contributions_qlime_sorted:
            print(f"Word: {word}, Importance: {weight}")
        
        #print("\n weights", clf.coef_[0])
        qlime_time = time.time() - start_qlime
        qlime_times.append(qlime_time)

        # top 5 (by absolute value)
        nonzero_indices = [
            (i, abs(contributions_qlime[i])) 
            for i in range(len(contributions_qlime))
        ]
        top_indices_qlime = sorted(nonzero_indices, key=lambda x: x[1], reverse=True)[:5]
        top_words_qlime = [
            vectorizer.get_feature_names_out()[i2]
            for (i2, val) in top_indices_qlime
        ]

        # measure overlap
        overlap = set(top_words_lime).intersection(set(top_words_qlime))
        overlaps.append(len(overlap))

    # Summary
    results = {
        "local_accuracy": np.mean(instance_local_accuracies),
        "lime_time_avg": round(np.mean(lime_times), 4),
        "qlime_time_avg": round(np.mean(qlime_times), 4),
        "overlap_avg": round(np.mean(overlaps), 4),
    }
    return results



# MAIN

In [54]:


if __name__ == "__main__":
    import pandas as pd

    # Parameter grid to systematically vary certain settings
    param_grid = {
        "num_samples": [500],
        "max_features": [20],
        "stopwords_option": [True],
        "lime_num_samples": [300],
        # Shots: None => analytic mode, 100 => finite sampling
        "shots": [100],
        "stop_words": ['english'],
        "n_test_explanations": [5]
    }

    combos = list(itertools.product(*param_grid.values()))
    all_results = []

    for combo in combos:
        (num_samples_, max_features_, stopwords_, lime_samps_, shots_, stop_words_, n_test_explanations_) = combo
        
        print("\n==================================")
        print(f"Running experiment with: "
              f"num_samples={num_samples_}, "
              f"max_features={max_features_}, "
              f"stopwords={stopwords_}, "
              f"lime_num_samples={lime_samps_}, "
              f"shots={shots_},"
              f"stop_words={stop_words_},"
              f"n_test_explanations={n_test_explanations_}")
        
        res = run_experiment(
            num_samples=num_samples_,
            max_features=max_features_,
            stopwords_option=stopwords_,
            lime_num_samples=lime_samps_,
            shots=shots_,
            stop_words=stop_words_,
            n_test_explanations=n_test_explanations_,
            
            
        )
        res_row = {
            "num_samples": num_samples_,
            "max_features": max_features_,
            "stopwords": stopwords_,
            "lime_num_samples": lime_samps_,
            "shots": shots_,
            "local_accuracy": res["local_accuracy"],
            "lime_time_avg": res["lime_time_avg"],
            "qlime_time_avg": res["qlime_time_avg"],
            "overlap_avg": res["overlap_avg"],
            "n_test_explanations": n_test_explanations_,
            "stop_words": stop_words_
        }
        print("Results =>", res_row)
        all_results.append(res_row)

    # Save results to CSV
    df = pd.DataFrame(all_results)
    df.to_csv("results_expanded_flips.csv", index=False)
    print("\nAll done! Saved results to 'results_expanded_flips.csv'.")


Running experiment with: num_samples=500, max_features=20, stopwords=True, lime_num_samples=300, shots=100,stop_words=english,n_test_explanations=5
No cached classifier found. Training a new one...
Cached classifier saved as cached_classifier_ns500_mf20_swTrue_logistic_classifier_seed42.pkl
text sample having been driven out of the house and into the theater by the sweltering heat, i could not have been more pleased. the road to perdition, directed by sam mendes (american beauty), is destined to become one of the greatest movies of all time. perhaps i'm just getting old; perhaps i've just seen the same themes recycled time and again. but this movie is indeed different.the story opens with young michael sullivan jr. facing out to the sea, contemplating the duality of his father's legacy -- one of the best men to ever live, one of the most evil. this duality snakes its way throughout the movie. the story revolves around crime boss john rooney (paul newman) and michael sullivan (tom hank